## PROTOTYPE Meridional Overturning in Potential Density Coordinates
here is my prototyped meridional overturning script.
this script is currently using the first 12 months of ecco data starting on Jan 16th, 1992
the demo data is stored in "./demo_nctiles_monthly/"

In [1]:
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import numpy as np
import xarray as xr


from xmitgcm import open_mdsdataset
import xmitgcm
import ecco_v4_py as ecco


from netCDF4 import Dataset

import seawater

from analysis_package import plotting_functions
from analysis_package import open_datasets
from analysis_package import derive_potential_density_values_TEST
from analysis_package import ecco_masks

from importlib import reload

# reload modules for prototyping...
ecco_masks = reload(ecco_masks)
plotting_functions = reload(plotting_functions)
open_datasets = reload(open_datasets)
derive_potential_density_values_TEST = reload(derive_potential_density_values_TEST)



In [2]:
######################################################################################################################
################################## DEFINE DATA FILES AND THEIR ASSOCIATED PATHS ######################################
######################################################################################################################

# complete_data_dir contains ECCO state data over the entire dataset time range (1992-2016, 288 monthly mean time steps)
complete_data_dir = "./nctiles_monthly/"
# dem_dir conatins ECCO state data truncated to the first 12 monthly mean time steps
demo_dir = "./demo_nctiles_monthly/"
data_dir = complete_data_dir

# UVELMASS is the ECCO eulerian flow velocity in the (native-grid)  x-direction
UVELMASS_var = "UVELMASS"
# VVELMASS is the ECCO eulerian flow velocity in the (native-grid) y-direction
VVELMASS_var = "VVELMASS"
# BOLUS_UVEL is the ECCO lagrangian flow velocity in the (native-grid)  x-direction
BOLUS_UVEL_var = "BOLUS_UVEL"
# BOLUS_VVEL is the ECCO lagrangian flow velocity in the (native-grid) y-direction
BOLUS_VVEL_var = "BOLUS_VVEL"

# RHOAnoma: insitu density anomaly
RHOAnoma_var_str = "RHOAnoma"
# PHIHYD: insitu pressure anomaly with respect to the depth integral of gravity and reference density (g*rho_reference)
PHIHYD_var_str = "PHIHYD"
# SALT: insitu salinity (psu)
SALT_var_str = "SALT"
# THETA: potential pressure (C)
THETA_var_str = "THETA"

# PDENS_U: Potential density interpolated to local grid cell "u" (i_g,j) points at sigma 2
PDENS_U_var_str = "PDENS_U"

# PDENS_V: Potential density interpolated to local grid cell "v" (i,j_g) points at sigma 2
PDENS_V_var_str = "PDENS_V"

In [3]:
######################################################################################################################
############################################ DEFINE TIMESLICE ########################################################
######################################################################################################################

time_slice = np.arange(0,12)
        

In [4]:
######################################################################################################################
################################################### LOAD GRID ########################################################
######################################################################################################################


grid_path = "./ecco_grid/ECCOv4r3_grid.nc"
grid = xr.open_dataset(grid_path)

In [5]:
######################################################################################################################
################################################### LOAD DATA ########################################################
######################################################################################################################

# load data files from central directory
UVELMASS_ds_raw = open_datasets.open_combine_raw_ECCO_tile_files(data_dir, UVELMASS_var, time_slice)
VVELMASS_ds_raw = open_datasets.open_combine_raw_ECCO_tile_files(data_dir, VVELMASS_var, time_slice)
BOLUS_UVEL_raw = open_datasets.open_combine_raw_ECCO_tile_files(data_dir, BOLUS_UVEL_var, time_slice,
                                                                rename_indices=False)
BOLUS_VVEL_raw = open_datasets.open_combine_raw_ECCO_tile_files(data_dir, BOLUS_VVEL_var, time_slice,
                                                                rename_indices=False)
PHIHYD_ds_raw = open_datasets.open_combine_raw_ECCO_tile_files(data_dir, PHIHYD_var_str, time_slice)
SALT_ds_raw = open_datasets.open_combine_raw_ECCO_tile_files(data_dir, SALT_var_str, time_slice)
THETA_ds_raw = open_datasets.open_combine_raw_ECCO_tile_files(data_dir, THETA_var_str, time_slice)

# set data file indecies starting from zero.
UVELMASS_ds_raw = UVELMASS_ds_raw.assign_coords(i_g=np.arange(0,90),j=np.arange(0,90),k=np.arange(0,50),
                                                time=time_slice)
VVELMASS_ds_raw = VVELMASS_ds_raw.assign_coords(i=np.arange(0,90),j_g=np.arange(0,90),k=np.arange(0,50),
                                                time=time_slice)
BOLUS_UVEL_raw = BOLUS_UVEL_raw.assign_coords(i_g=np.arange(0,90),j=np.arange(0,90),k=np.arange(0,50),
                                              time=time_slice)
BOLUS_VVEL_raw = BOLUS_VVEL_raw.assign_coords(i=np.arange(0,90),j_g=np.arange(0,90),k=np.arange(0,50),
                                              time=time_slice)
PHIHYD_ds_raw = PHIHYD_ds_raw.assign_coords(i=np.arange(0,90),j=np.arange(0,90),k=np.arange(0,50),
                                            time=time_slice)
SALT_ds_raw = SALT_ds_raw.assign_coords(i=np.arange(0,90),j=np.arange(0,90),k=np.arange(0,50),
                                        time=time_slice)
THETA_ds_raw = THETA_ds_raw.assign_coords(i=np.arange(0,90),j=np.arange(0,90),k=np.arange(0,50),
                                          time=time_slice)




Loaded UVELMASS over time slice  

Loaded VVELMASS over time slice  

Loaded BOLUS_UVEL over time slice  

Loaded BOLUS_VVEL over time slice  

Loaded PHIHYD over time slice  

Loaded SALT over time slice  

Loaded THETA over time slice  



In [6]:
######################################################################################################################
################################## CALCULATE POTENTIAL DENSITY #######################################################
######################################################################################################################


# calculate potential density and in situ pressure
PDENS_ds, P_INSITY_ds = derive_potential_density_values_TEST.make_potential_density_dataset(PHIHYD_ds_raw, 
                                                                                            SALT_ds_raw, 
                                                                                            THETA_ds_raw, 
                                                                                            time_slice, 
                                                                                            ref_pressure=2000.)
# set data file indecies starting from zero.
PDENS_ds = PDENS_ds.assign_coords(i=np.arange(0,90),j=np.arange(0,90),k=np.arange(0,50))

# calculate potential density and in situ pressure
PDENS_U_ds_raw = open_datasets.open_combine_raw_ECCO_tile_files(data_dir,PDENS_U_var_str,time_slice,rename_indices=False)
# set data file indecies starting from zero.
PDENS_U_ds = PDENS_U_ds_raw.assign_coords(i=np.arange(0,90),j=np.arange(0,90),k=np.arange(0,50),time=time_slice)
# calculate potential density and in situ pressure
PDENS_V_ds_raw = open_datasets.open_combine_raw_ECCO_tile_files(data_dir,PDENS_V_var_str,time_slice,rename_indices=False)
# set data file indecies starting from zero.
PDENS_V_ds = PDENS_V_ds_raw.assign_coords(i=np.arange(0,90),j=np.arange(0,90),k=np.arange(0,50),time=time_slice)

Loaded PDENS_U over time slice  

Loaded PDENS_V over time slice  



In [7]:
######################################################################################################################
############################################# CREATE DOMAIN MASKS ####################################################
######################################################################################################################

maskW = (UVELMASS_ds_raw.UVELMASS.isel(k=0,time=1)*0 + 1.)
maskS = (VVELMASS_ds_raw.VVELMASS.isel(k=0,time=1)*0 + 1.)
maskC = (PDENS_ds.PDENS.isel(k=0,time=0)*0 + 1.)


southern_ocean_mask_W, southern_ocean_mask_S, so_atl_basin_mask_W, so_atl_basin_mask_S, so_indpac_basin_mask_W, so_indpac_basin_mask_S = ecco_masks.get_basin_masks(maskW, maskS, maskC)





In [ ]:
######################################################################################################################
############################### CALCULATE TRANSPORT IN POTENTIAL DENSITY SPACE #######################################
######################################################################################################################


cds = grid.coords.to_dataset()
grid_xmitgcm = ecco.ecco_utils.get_llc_grid(cds)

transport_x = (UVELMASS_ds_raw["UVELMASS"]*grid["drF"]*grid["dyG"] 
               + BOLUS_UVEL_raw["bolus_uvel"]*grid["drF"]*grid["dyG"]*grid["hFacW"])*so_indpac_basin_mask_W
transport_y = (VVELMASS_ds_raw["VVELMASS"]*grid["drF"]*grid["dxG"] 
               + BOLUS_VVEL_raw["bolus_vvel"]*grid["drF"]*grid["dxG"]*grid["hFacS"])*so_indpac_basin_mask_S

# create infrastructure for integrating in depth space

lat_vals = np.arange(-88,88)

# create an empty array with a stretched depth dimension
# Set the coordinates of the stretched depth dimension to potential density values..
# add padding to either end of the pot. density coordinates
# just trying with slightly coarser resolution 
#(what pot density resolution is valid in this case?)
pot_dens_coord = np.arange(1032.1,1036.51,0.2)

pot_dens_coord = np.concatenate((np.asarray([1000.]),pot_dens_coord, np.arange(1036.7,1037.31,0.05)))

# set dimensions based on input dataset with modified vertical level spacing..
pot_dens_dims = (len(time_slice),
                 len(pot_dens_coord),
                 len(lat_vals))

empty_pot_coords_data = np.zeros(pot_dens_dims)
# trying to make this as general as possible, but need to keep an eye on this..
new_coords = [time_slice, pot_dens_coord, lat_vals]
new_dims = ["time", "pot_rho", "lat"]

pot_dens_array_x = PDENS_U_ds.PDENS.copy(deep=True)*so_indpac_basin_mask_W
pot_dens_array_y = PDENS_V_ds.PDENS.copy(deep=True)*so_indpac_basin_mask_S

depth_integrated_pdens_transport = xr.DataArray(data=empty_pot_coords_data,coords=new_coords,dims=new_dims)
depth_integrated_pdens_transport.load()
depth_integrated_pdens_transport_latx = depth_integrated_pdens_transport.copy(deep=True)
depth_integrated_pdens_transport_latx.load()
depth_integrated_pdens_transport_laty = depth_integrated_pdens_transport.copy(deep=True)
depth_integrated_pdens_transport_laty.load()

depth_integrated_x_interp_results = depth_integrated_pdens_transport.copy(deep=True)
depth_integrated_x_interp_results.load()
depth_integrated_y_interp_results = depth_integrated_pdens_transport.copy(deep=True)
depth_integrated_y_interp_results.load()

depth_integrated_x_pdens_transport_no_interp = depth_integrated_pdens_transport.copy(deep=True)
depth_integrated_x_pdens_transport_no_interp.load()
depth_integrated_y_pdens_transport_no_interp = depth_integrated_pdens_transport.copy(deep=True)
depth_integrated_y_pdens_transport_no_interp.load()


for density in pot_dens_coord:
    print("Started " + str(density) + " surface") 
    potdens_stencil_x_0 = pot_dens_array_x > density
    potdens_stencil_y_0 = pot_dens_array_y > density
    # this step is critical to remove low density anomalies in the deep ocean from stencil...
    potdens_stencil_x = potdens_stencil_x_0.cumsum(dim="k") > 0
    potdens_stencil_y = potdens_stencil_y_0.cumsum(dim="k") > 0
    print("got to checkpoint 0")

    
    ##################################################################################################################
    ###########################################     START INTERPOLATION    ###########################################
    ##################################################################################################################
    
    # set end-appended value equal to 1 for subtraction step..
    potdens_stencil_x_shifted_up_one_cell = xr.concat((potdens_stencil_x.isel(k=slice(1,50)),
                                                       potdens_stencil_x.isel(k=49)),
                                                      dim="k").assign_coords(k=np.arange(0,50))
    potdens_stencil_y_shifted_up_one_cell = xr.concat((potdens_stencil_y.isel(k=slice(1,50)),
                                                       potdens_stencil_y.isel(k=49)),
                                                      dim="k").assign_coords(k=np.arange(0,50))
    potdens_stencil_x_shifted_down_one_cell = xr.concat((potdens_stencil_x.isel(k=0)*0,
                                                         potdens_stencil_x.isel(k=slice(0,49))),
                                                        dim="k").assign_coords(k=np.arange(0,50))
    potdens_stencil_y_shifted_down_one_cell = xr.concat((potdens_stencil_y.isel(k=0)*0,
                                                         potdens_stencil_y.isel(k=slice(0,49))),
                                                        dim="k").assign_coords(k=np.arange(0,50))

    potdens_stencil_x_one_above_top_level = potdens_stencil_x_shifted_up_one_cell*1 - potdens_stencil_x*1
    potdens_stencil_y_one_above_top_level = potdens_stencil_y_shifted_up_one_cell*1 - potdens_stencil_y*1
    # get rid of trailing negative values that occur at the ocean's bottom boundary..
    potdens_stencil_x_one_above_top_level = potdens_stencil_x_one_above_top_level.where(potdens_stencil_x_one_above_top_level > 0,
                                                                                        other=0)
    potdens_stencil_y_one_above_top_level = potdens_stencil_y_one_above_top_level.where(potdens_stencil_y_one_above_top_level > 0,
                                                                                        other=0)

    potdens_stencil_x_top_level = potdens_stencil_x*1 - potdens_stencil_x_shifted_down_one_cell*1
    potdens_stencil_y_top_level = potdens_stencil_y*1 - potdens_stencil_y_shifted_down_one_cell*1
    # turn zeros into nans..
    # NOTE SOMETIMES YOU GET PROTRUSIONS OF DENSITY ANOMALIES THAT SEEM TO CREATE TWO DENSITY SURFACES, LEADING TO A VALUE OF
    # 2 IN THE STENCIL.. I eliminated this using "potdens_stencil_x = potdens_stencil_x_0.cumsum(dim="k") > 0" a couple lines above.
    potdens_stencil_x_top_level = potdens_stencil_x_top_level.where(potdens_stencil_x_top_level > 0, other=np.nan)
    potdens_stencil_y_top_level = potdens_stencil_y_top_level.where(potdens_stencil_y_top_level > 0, other=np.nan)
    potdens_stencil_x_one_above_top_level = potdens_stencil_x_one_above_top_level.where(potdens_stencil_x_one_above_top_level > 0,
                                                                                        other=np.nan)
    potdens_stencil_y_one_above_top_level = potdens_stencil_y_one_above_top_level.where(potdens_stencil_y_one_above_top_level > 0,
                                                                                        other=np.nan)
    # multiply depth values by -1 to make them positive..
    depth_above_x_top_level_raw = (-1*potdens_stencil_x_one_above_top_level.fillna(0)*transport_x.Z).sum(dim="k")
    depth_x_top_level_raw = (-1*potdens_stencil_x_top_level.fillna(0)*transport_x.Z).sum(dim="k",skipna=True)
    depth_above_y_top_level_raw = (-1*potdens_stencil_y_one_above_top_level.fillna(0)*transport_y.Z).sum(dim="k",skipna=True)
    depth_y_top_level_raw = (-1*potdens_stencil_y_top_level.fillna(0)*transport_y.Z).sum(dim="k",skipna=True)
    # turn zeros into nans..
    depth_above_x_top_level = depth_above_x_top_level_raw.where(depth_above_x_top_level_raw > 0, other=np.nan)
    depth_x_top_level = depth_x_top_level_raw.where(depth_x_top_level_raw > 0, other=np.nan)
    depth_above_y_top_level = depth_above_y_top_level_raw.where(depth_above_y_top_level_raw > 0, other=np.nan)
    depth_y_top_level = depth_y_top_level_raw.where(depth_y_top_level_raw > 0, other=np.nan)

    thickness_above_x_top_level_raw = (potdens_stencil_x_one_above_top_level.fillna(0)*grid["drF"]).sum(dim="k")
    thickness_x_top_level_raw = (potdens_stencil_x_top_level.fillna(0)*grid["drF"]).sum(dim="k")
    thickness_above_y_top_level_raw = (potdens_stencil_y_one_above_top_level.fillna(0)*grid["drF"]).sum(dim="k")
    thickness_y_top_level_raw = (potdens_stencil_y_top_level.fillna(0)*grid["drF"]).sum(dim="k")
    # turn zeros into nans..
    thickness_above_x_top_level = thickness_above_x_top_level_raw.where(thickness_above_x_top_level_raw > 0, other=np.nan)
    thickness_x_top_level = thickness_x_top_level_raw.where(thickness_x_top_level_raw > 0, other=np.nan)
    thickness_above_y_top_level = thickness_above_y_top_level_raw.where(thickness_above_y_top_level_raw > 0, other=np.nan)
    thickness_y_top_level = thickness_y_top_level_raw.where(thickness_y_top_level_raw > 0, other=np.nan)    
    
    potdens_above_x_top_level = (potdens_stencil_x_one_above_top_level.fillna(0)*pot_dens_array_x.fillna(0)).sum(dim="k")
    potdens_x_top_level = (potdens_stencil_x_top_level.fillna(0)*pot_dens_array_x.fillna(0)).sum(dim="k")
    potdens_above_y_top_level = (potdens_stencil_y_one_above_top_level.fillna(0)*pot_dens_array_y.fillna(0)).sum(dim="k")
    potdens_y_top_level = (potdens_stencil_y_top_level.fillna(0)*pot_dens_array_y.fillna(0)).sum(dim="k")
    # turn zeros into nans..
    potdens_above_x_top_level = potdens_above_x_top_level.where(potdens_above_x_top_level > 0, other=np.nan)
    potdens_x_top_level = potdens_x_top_level.where(potdens_x_top_level > 0, other=np.nan)
    potdens_above_y_top_level = potdens_above_y_top_level.where(potdens_above_y_top_level > 0, other=np.nan)
    potdens_y_top_level = potdens_y_top_level.where(potdens_y_top_level > 0, other=np.nan)
    
    print("got to checkpoint 1")
    transport_above_x_top_level = (potdens_stencil_x_one_above_top_level.fillna(0)*transport_x).sum(dim="k")
    transport_x_top_level = (potdens_stencil_x_top_level.fillna(0)*transport_x).sum(dim="k")
    transport_above_y_top_level = (potdens_stencil_y_one_above_top_level.fillna(0)*transport_y).sum(dim="k")
    transport_y_top_level = (potdens_stencil_y_top_level.fillna(0)*transport_y).sum(dim="k")
    
    transport_above_x_top_level = transport_above_x_top_level.where(transport_above_x_top_level !=0, other=np.nan)    
    transport_x_top_level = transport_x_top_level.where(transport_x_top_level != 0, other=np.nan)
    transport_above_y_top_level = transport_above_y_top_level.where(transport_above_y_top_level !=0, other=np.nan)
    transport_y_top_level = transport_y_top_level.where(transport_y_top_level !=0, other=np.nan)
    
    depth_potdens_slope_x = (depth_above_x_top_level - depth_x_top_level)/(potdens_above_x_top_level - potdens_x_top_level)                                                         
    depth_potdens_slope_y = (depth_above_y_top_level - depth_y_top_level)/(potdens_above_y_top_level - potdens_y_top_level)
    
    # this is an issue... need to account for those low desnity protrusions..
    h_array_x_0 = (density - potdens_x_top_level)*depth_potdens_slope_x
    h_array_x = h_array_x_0.where(h_array_x_0 < 0, other=0 )*-1
    h_array_y_0 = (density - potdens_y_top_level)*depth_potdens_slope_y 
    h_array_y = h_array_y_0.where(h_array_y_0 < 0, other=0 )*-1
    
    print("got to checkpoint 2")
    half_thickness_x_top_level = (potdens_stencil_x_top_level.fillna(0)*grid.drF/2.).sum(dim="k")
    half_thickness_y_top_level = (potdens_stencil_y_top_level.fillna(0)*grid.drF/2.).sum(dim="k")
    half_thickness_x_one_above_top_level = (potdens_stencil_x_one_above_top_level.fillna(0)*grid.drF/2.).sum(dim="k")
    half_thickness_y_one_above_top_level = (potdens_stencil_y_one_above_top_level.fillna(0)*grid.drF/2.).sum(dim="k")
    
    overshoot_x_0 = h_array_x - half_thickness_x_top_level
    overshoot_x = overshoot_x_0.where(overshoot_x_0 > 0).fillna(0)
    undershoot_x = overshoot_x_0.where(overshoot_x_0 <= 0).fillna(0)
    overshoot_y_0 = h_array_y - half_thickness_y_top_level
    overshoot_y = overshoot_y_0.where(overshoot_y_0 > 0).fillna(0)
    undershoot_y = overshoot_y_0.where(overshoot_y_0 <= 0).fillna(0)
    
    percent_top_filled_x = (half_thickness_x_top_level + undershoot_x)/(half_thickness_x_top_level*2)
    percent_top_filled_y = (half_thickness_y_top_level + undershoot_y)/(half_thickness_y_top_level*2)
    
    percent_one_above_top_filled_x = overshoot_x/(half_thickness_x_one_above_top_level*2)
    percent_one_above_top_filled_y = overshoot_y/(half_thickness_y_one_above_top_level*2)  
    
    trsp_interpolated_x = (percent_top_filled_x*transport_x_top_level).fillna(0) + (percent_one_above_top_filled_x*transport_above_x_top_level).fillna(0)
    trsp_interpolated_y = (percent_top_filled_y*transport_y_top_level).fillna(0) + (percent_one_above_top_filled_y*transport_above_y_top_level).fillna(0)
    
    # "transport_integral_x/y" is the vertical sum of the interpolated grid cell tranposrt
    trsp_interpolated_x.load()
    trsp_interpolated_y.load()
    
    ##################################################################################################################
    ###########################################     END INTERPOLATION    #############################################
    ##################################################################################################################   
    
    
    # split the top cell in half since we are putting it into the interpolation,
    # but only in cases where there actually is a cell above it.
    depth_integrated_trsp_x = transport_x*(potdens_stencil_x.where(potdens_stencil_x>0,other=np.nan)) - (transport_x*potdens_stencil_x_top_level/2.).fillna(0)
    depth_integrated_trsp_x.load()
    depth_integrated_trsp_x = depth_integrated_trsp_x.sum(dim='k') + trsp_interpolated_x.fillna(0)
    
    depth_integrated_trsp_y = transport_y*(potdens_stencil_y.where(potdens_stencil_y>0,other=np.nan)) - (transport_y*potdens_stencil_y_top_level/2.).fillna(0)
    depth_integrated_trsp_y.load()
    depth_integrated_trsp_y = depth_integrated_trsp_y.sum(dim='k') + trsp_interpolated_y.fillna(0)
    
    depth_integrated_trsp_x_no_interp = (transport_x*potdens_stencil_x).sum(dim='k')
    depth_integrated_trsp_x_no_interp.load()
    depth_integrated_trsp_y_no_interp = (transport_y*potdens_stencil_y).sum(dim='k')
    depth_integrated_trsp_y_no_interp.load()
                                          
                                           
    print('starting lat-band filtering')
    for lat in lat_vals:
        # Compute mask for particular latitude band
        print(str(lat)+' ',end='')
        # since transport values are in native grid coordaintes you need to combine the sum of the transports in 
        # the x and y direction and vis versa for tiles 0-5 and 7-12, respectively
        lat_maskX, lat_maskY = ecco.vector_calc.get_latitude_masks(lat, cds['YC'], grid_xmitgcm)
       
        # Sum horizontally
        lat_trsp_x = (depth_integrated_trsp_x * lat_maskX).sum(dim=['i_g','j','tile'],skipna=True)
        lat_trsp_y = (depth_integrated_trsp_y * lat_maskY).sum(dim=['i','j_g','tile'],skipna=True)
        lat_trsp_x_interp = (trsp_interpolated_x * lat_maskX).sum(dim=['i_g','j','tile'],skipna=True)
        lat_trsp_y_interp = (trsp_interpolated_y * lat_maskY).sum(dim=['i','j_g','tile'],skipna=True)
        # data for overturning with no interpolation..
        lat_trsp_x_no_interp = (depth_integrated_trsp_x_no_interp * lat_maskX).sum(dim=['i_g','j','tile'],skipna=True)
        lat_trsp_y_no_interp = (depth_integrated_trsp_y_no_interp * lat_maskY).sum(dim=['i','j_g','tile'],skipna=True)
        
        depth_integrated_pdens_transport_latx.loc[{'lat':lat,'pot_rho':density}] = lat_trsp_x
        depth_integrated_pdens_transport_laty.loc[{'lat':lat,'pot_rho':density}] = lat_trsp_y
        depth_integrated_x_interp_results.loc[{'lat':lat,'pot_rho':density}] = lat_trsp_x_interp
        depth_integrated_y_interp_results.loc[{'lat':lat,'pot_rho':density}] = lat_trsp_y_interp
        depth_integrated_x_pdens_transport_no_interp.loc[{'lat':lat,'pot_rho':density}] = lat_trsp_x_no_interp
        depth_integrated_y_pdens_transport_no_interp.loc[{'lat':lat,'pot_rho':density}] = lat_trsp_y_no_interp                                

    print("\n")
    
    



## OUTPUT PLOTS

In [ ]:
min_lat = 0
max_lat = 50
d_levels = np.arange(0,50)

pvmin = -2e7
pvmax = 2e7

plt.figure(figsize=(20,10))
#vmin = depth_integrated_pdens_transport.min()
#vmax = depth_integrated_pdens_transport.max()
plt.contourf(depth_integrated_pdens_transport_laty.lat[min_lat:max_lat],
             pot_dens_coord[1:],
             -1*depth_integrated_pdens_transport_laty.mean(dim='time')[1:,min_lat:max_lat]-1*depth_integrated_pdens_transport_latx.mean(dim='time')[1:,min_lat:max_lat],
             30,
             cmap='bwr',
            vmin=pvmin,
            vmax=pvmax)
cbar = plt.colorbar()
cbar.set_label("Transport (Sv)",rotation=270)
plt.title("Meridional overturning results with interpolation",fontname='times new roman',fontsize=24)
plt.xlabel("Latitude (deg)",fontname='Times New Roman',fontsize=16)
plt.ylabel("$\sigma_{2}$ (kg/m^3)",fontname='Times New Roman',fontsize=16)
cbar.set_label("Transport (Sv)",fontname='Times New Roman',fontsize=16,rotation=270)
plt.grid()
plt.gca().invert_yaxis()
plt.savefig("./figures/southern_ocean_indopacific_overturning_with_interp")
plt.show()
plt.close()

plt.figure(figsize=(20,10))
plt.contourf(depth_integrated_pdens_transport_laty.lat[min_lat:max_lat],
             pot_dens_coord[1:],
             -1*depth_integrated_pdens_transport_latx.mean(dim='time')[1:,min_lat:max_lat] -1*depth_integrated_pdens_transport_laty.mean(dim='time')[1:,min_lat:max_lat]    
             -(-1*depth_integrated_x_interp_results.mean(dim='time')[1:,min_lat:max_lat]-1*depth_integrated_y_interp_results.mean(dim='time')[1:,min_lat:max_lat]),
             30,
             cmap='bwr',
             vmin=pvmin,
             vmax=pvmax)
cbar = plt.colorbar()
plt.title("Meridional overturning results without interpolation",fontname='Times New Roman',fontsize=24)
plt.xlabel("Latitude (deg)",fontname='Times New Roman',fontsize=16)
plt.ylabel("$\sigma_{2}$ (kg/m^3)",fontname='Times New Roman',fontsize=16)
cbar.set_label("Transport (Sv)",fontname='Times New Roman',fontsize=16,rotation=270)
plt.grid()
plt.gca().invert_yaxis()
plt.savefig("./figures/southern_ocean_indopacific_overturning_NO_interp")
plt.show()
plt.close()


plt.figure(figsize=(20,10))
plt.contourf(depth_integrated_pdens_transport_laty.lat[min_lat:max_lat],
             pot_dens_coord[1:],
             -(-1*depth_integrated_x_interp_results.mean(dim='time')[1:,min_lat:max_lat]-1*depth_integrated_y_interp_results.mean(dim='time')[1:,min_lat:max_lat]),
             30,
             cmap='bwr',
             vmin=pvmin,
             vmax=pvmax)
cbar = plt.colorbar()
cbar.set_label("Transport (Sv)",rotation=270)
plt.title("Interpolation correction values",fontname='Times New Roman',fontsize=24)
plt.xlabel("Latitude (deg)",fontname='Times New Roman',fontsize=16)
plt.ylabel("$\sigma_{2}$ (kg/m^3)",fontname='Times New Roman',fontsize=16)
plt.grid()
plt.gca().invert_yaxis()
plt.savefig("./figures/southern_ocean_indopacific_overturning_interp_Correction")
plt.show()
plt.close()
